In [1]:
import os
import polars as pl

In [63]:
from src.pocet_stran import pocet_stran

In [85]:
df = pl.read_parquet(os.path.join("data","cnb_ceska_proza.parquet"))

In [87]:
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","020.parquet")), on="001", how="left")

In [89]:
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","300.parquet")), on="001", how="left")
df = df.with_columns(pl.col('300_a').map_elements(pocet_stran, return_dtype=int).alias('stran'))

In [90]:
df.sample()

rok,titul,autorstvo,autorstvo_kod,001,020_q,020_c,020_a,020_z,300_a,300_b,300_c,300_e,300_f,300_3,stran
f64,str,str,str,str,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],str,str,i64
2020.0,"""Kvadriga""","""Stošková Lucie""","""xx0255773""","""nkc20213302452""","[""(vázáno)""]",null,"[""978-80-88049-89-0""]",null,"[""528 stran :""]","[""ilustrace ;""]","[""22 cm""]",null,null,null,528


In [41]:
def cena(retezec):
    try:
        return int(retezec.split("Kč")[1].split(',')[0].strip())
    except:
        return None

In [93]:
df = df.explode("020_c").filter(pl.col('020_c').is_not_null()).with_columns(pl.col("020_c").map_elements(lambda x: cena(x)).alias('cena'))

sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredict

In [121]:
df.filter(pl.col('stran') > 250).group_by('rok').agg(pl.col('cena').mean()).sort(by='rok').tail(10)

rok,cena
f64,f64
2016.0,294.205128
2017.0,337.709091
2018.0,313.358209
2019.0,340.372093
2020.0,364.673267
2021.0,344.756303
2022.0,408.39823
2023.0,421.67033
2024.0,408.852941


In [67]:
import altair as alt

In [101]:
do_grafu = df.filter(pl.col('rok') == 2024).unique(subset=['titul','autorstvo']).select(pl.col(['stran','cena']))
do_grafu

stran,cena
i64,i64
189,325
409,459
199,349
119,329
197,295
…,…
276,399
58,329
93,299


In [115]:
zaklad = alt.Chart(do_grafu.to_pandas()).mark_circle().encode(
    alt.X('stran:Q'),
    alt.Y('cena:Q')
)
cara = alt.Chart(
    do_grafu.to_pandas()
).transform_regression(
    'stran', 'cena'
).mark_line(color='#445B78').encode(
    alt.X('stran:Q'),
    alt.Y('cena:Q')
)

vysledek = (zaklad + cara)

vysledek

alt.LayerChart(...)

In [ ]:
do_grafu.filter(pl.col('stran').is_between(200,220)).select(pl.col('cena')).median()

In [117]:
do_grafu.filter(pl.col('stran').is_between(200,220)).select(pl.col('cena')).median()

cena
f64
349.0


In [123]:
do_grafu.filter(pl.col('stran').is_between(400,420)).select(pl.col('cena')).median()

cena
f64
449.0


In [81]:
df.filter(pl.col("rok") == 2024).unique(subset=["titul","autorstvo"])

ColumnNotFoundError: "titul" not found

In [79]:
df.group_by('rok').agg(pl.col('cena').median()).sort(by='rok').tail(10)

rok,cena
f64,f64
2016.0,217.9
2017.0,240.336508
2018.0,241.036697
2019.0,256.152968
2020.0,277.012195
2021.0,282.81978
2022.0,317.807143
2023.0,315.678019
2024.0,323.872781


In [53]:
df.filter(pl.col('020_c').is_not_null()).with_columns(pl.col("020_c").map_elements(lambda x: cena(x)).alias('cena')).group_by('rok').len().sort(by="rok")

C:\Users\micha\AppData\Local\Temp\ipykernel_23460\5590320.py:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  df.filter(pl.col('020_c').is_not_null()).with_columns(pl.col("020_c").map_elements(lambda x: cena(x)).alias('cena')).group_by('rok').len().sort(by="rok")


rok,len
f64,u32
1879.0,1
1885.0,1
1893.0,1
1901.0,3
1903.0,2
…,…
2021.0,469
2022.0,425
2023.0,328


In [55]:
df.group_by("rok").len().sort(by="rok")

rok,len
f64,u32
1804.0,2
1807.0,2
1808.0,1
1810.0,2
1813.0,3
…,…
2021.0,1505
2022.0,1503
2023.0,1307
